# LangChain サンプル 5: LangChain Expression Language (LCEL)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックのサンプルは、LangChain のドキュメント [How to migrate chains to LCEL](https://python.langchain.com/v0.2/docs/how_to/migrate_chains/#llmchain) の内容をベースに Amazon Bedrock 向けに変更したものです。

### LLMChain のサンプル

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock


chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id とする
)

prompt = ChatPromptTemplate.from_messages(
    [("user", "{country} の首都は？")],
)

chain = LLMChain(llm=chat, prompt=prompt)

result = chain.invoke({"country": "日本"})

print(result)

### LLMChain のサンプルを LCEL で書き直したサンプル

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id とする
)

prompt = ChatPromptTemplate.from_messages(
    [("user", "{country} の首都は？")],
)

chain = prompt | chat | StrOutputParser()

result = chain.invoke({"country": "日本"})

print(result)

### ConversationChain のサンプル

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id とする
)

template = """
あなたは海賊です。次の質問にできるだけ正確に答えてください。
Chat history: {history}
Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)

memory = ConversationBufferMemory()

chain = ConversationChain(
    llm=chat,
    memory=memory,
    prompt=prompt,
)

result = chain.invoke({"input": "ご機嫌いかがですか?日本語で答えてくださいね。"})

print(result)

### ConversationChain のサンプルを LCEL で書き直したサンプル

### 注意：このサンプルは langchain_core の v0.2 が必要です。この環境では実行する必要はありません。

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id とする
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは海賊です。次の質問にできるだけ正確に答えてください。"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history

chain = prompt | chat | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)

wrapped_chain.invoke({"input": "how are you?"})

result = wrapped_chain.invoke({"input": "ご機嫌いかがですか?日本語で答えてくださいね。"})

print(result)

### RetrievalQA のサンプル

In [ ]:
!pip install langchain_community
!pip install faiss-cpu
!pip install langchainhub

In [ ]:
# Load docs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_aws.chat_models import ChatBedrock
from langchain_aws.embeddings import BedrockEmbeddings

from langchain import hub
from langchain.chains import RetrievalQA

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Embed model
embeddings = BedrockEmbeddings( # BedrockEmbeddingsを初期化する
    model_id = "amazon.titan-embed-text-v1" # Bedrock では model_id にする
)

# Store splits
vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)

# LLM
chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id とする
)

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_llm(
    chat, retriever=vectorstore.as_retriever(), prompt=prompt
)

result = qa_chain.invoke("autonomous agents とは何ですか？日本語で回答してください。")

print(result)

### RetrievalQA のサンプルを LCEL で書き直したサンプル

In [ ]:
# Load docs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_aws.chat_models import ChatBedrock
from langchain_aws.embeddings import BedrockEmbeddings

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Embed model
embeddings = BedrockEmbeddings( # BedrockEmbeddingsを初期化する
    model_id = "amazon.titan-embed-text-v1" # Bedrock では model_id にする
)

# Store splits
vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)

# LLM
chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id とする
)

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vectorstore.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | chat
    | StrOutputParser()
)

result = qa_chain.invoke("autonomous agents とは何ですか？日本語で回答してください。")

print(result)